## Importando o DuckDB e criando uma tabela nova

In [1]:
import duckdb

#criando um database para o arquivo parquet de exemplo
database_path = "flights.db"

con = duckdb.connect(database_path)

### filghts.db criado. Vamos rodar uma query para salvar os dados do arquivo parquet de exemplo nesse db

In [2]:
parquet_file_path = "flights.parquet"

con.execute(f"CREATE TABLE flights AS SELECT * FROM '{parquet_file_path}'")
con.commit()

### Verificando se tudo foi criado corretamente

In [3]:
con.execute('SHOW TABLES').fetchall()

[('flights',)]

### Tudo criado corretamente. Vamos visualizar as infos da tabela

In [4]:
df = con.execute("SELECT * FROM flights").fetchdf()
df.describe().T

,count,mean,min,25%,50%,75%,max,std
FL_DATE,1000000,2006-01-28 16:45:33.379200,2006-01-01 00:00:00,2006-01-14 00:00:00,2006-01-28 00:00:00,2006-02-13 00:00:00,2006-02-28 00:00:00,NaN
DEP_DELAY,1000000.0,8.651135,-1197.0,-4.0,0.0,8.0,1379.0,31.167477
ARR_DELAY,1000000.0,6.400658,-197.0,-10.0,-2.0,11.0,1374.0,33.939187
AIR_TIME,1000000.0,105.81399,-2.0,55.0,87.0,136.0,877.0,69.33634
DISTANCE,1000000.0,741.258545,30.0,325.0,589.0,984.0,4962.0,573.779372
DEP_TIME,1000000.0,13.62196,0.0,9.583333,13.583333,17.5,23.983334,4.737427
ARR_TIME,1000000.0,15.134593,0.0,11.283334,15.383333,19.233334,23.983334,4.966053


### Criando novos dados artificiais pra testar a inserção deles na tabela

In [5]:
import numpy as np
import pandas as pd

QTD_DE_LINHAS_NOVAS = 23

new_data = pd.DataFrame({
    'FL_DATE': pd.to_datetime(np.random.choice(pd.date_range('2023-01-01', '2023-12-31'), QTD_DE_LINHAS_NOVAS)),
    'DEP_DELAY': np.random.randint(-1200, 1401, QTD_DE_LINHAS_NOVAS),
    'ARR_DELAY': np.random.randint(-200, 1401, QTD_DE_LINHAS_NOVAS),
    'AIR_TIME': np.random.randint(-2, 901, QTD_DE_LINHAS_NOVAS),
    'DISTANCE': np.random.randint(30, 5001, QTD_DE_LINHAS_NOVAS),
    'DEP_TIME': np.random.randint(0, 25, QTD_DE_LINHAS_NOVAS),
    'ARR_TIME': np.random.randint(0, 25, QTD_DE_LINHAS_NOVAS)
})
new_data.head()

,FL_DATE,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,DEP_TIME,ARR_TIME
0,2023-01-22,839,343,677,2631,4,23
1,2023-03-12,-36,1113,362,2745,0,22
2,2023-12-18,1160,106,287,1432,19,11
3,2023-01-13,342,1309,402,816,17,3
4,2023-05-08,-1001,589,72,4842,19,17


In [6]:
con.execute("INSERT INTO flights SELECT * FROM new_data")
con.commit()
con.close()

### Vamos refazer a conexão e ver se as linhas foram adicionadas corretamente

In [8]:
con = duckdb.connect(database_path)
con.execute("SELECT * FROM flights").fetchdf()

,FL_DATE,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,DEP_TIME,ARR_TIME
0,2006-01-01,5,19,350,2475,9.083333,12.483334
1,2006-01-02,167,216,343,2475,11.783334,15.766666
2,2006-01-03,-7,-2,344,2475,8.883333,12.133333
3,2006-01-04,-5,-13,331,2475,8.916667,11.950000
4,2006-01-05,-3,-17,321,2475,8.950000,11.883333
...,...,...,...,...,...,...,...
1000018,2023-09-26,1005,315,115,3568,17.000000,24.000000
1000019,2023-08-31,787,-40,416,818,18.000000,21.000000
1000020,2023-01-22,224,1242,612,3884,1.000000,18.000000
1000021,2023-10-20,-497,289,172,307,15.000000,14.000000
